# Sarvam AI Pipeline Notebook 📓  
**STT → Translation → TTS** (with optional Transliteration)  
*Generated on 2025-07-04*  

This Colab/Jupyter notebook demonstrates:  

1. **Speech‑to‑Text (STT)** using the `saarika:v2.5` model  
2. **Text Translation** using `sarvam‑translate`  
3. **Text‑to‑Speech (TTS)** with the **Bulbul** model  
4. (Optional) **Transliteration** examples (Romanisation ↔ Indic, numeral options)  

> 👉 Replace placeholders like `YOUR_SARVAM_API_KEY` with your real key.  


In [1]:
!uv add sarvamai

Resolved 80 packages in 81ms
Audited 74 packages in 0.14ms


In [2]:
from sarvamai import SarvamAI
from sarvamai.play import play, save
import os, sys, json

## 🔑 Set your API key

In [3]:
from dotenv import load_dotenv
load_dotenv()
SARVAM_API_KEY = os.getenv("SARVAM_API_KEY") or "YOUR_SARVAM_API_KEY"  # ⬅️ EDIT ME
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

## 🎙 Upload or provide audio file (.wav / .mp3)

In [4]:
def get_audio_file():
    supported = ['.wav', '.mp3']
    if 'google.colab' in sys.modules:
        from google.colab import files
        uploaded = files.upload()
        if not uploaded:
            return None
        path = list(uploaded.keys())[0]
    else:
        path = input("Enter path to .wav / .mp3 file: ").strip()
    if not os.path.exists(path):
        print("❌ File not found:", path); return None
    if os.path.splitext(path)[1].lower() not in supported:
        print("❌ Unsupported format."); return None
    print("✅ Using audio:", path)
    return path

audio_file_path = get_audio_file()

✅ Using audio: ./sample_hindi.wav


## 📝 Step 1 – Speech‑to‑Text (Saarika v2.5)

In [10]:
%%time
if audio_file_path:
    with open(audio_file_path, 'rb') as f:
        stt_resp = client.speech_to_text.transcribe(
            file=f,
            model='saarika:v2.5',
            language_code='unknown'  # auto‑detect
        )
    print("🗣️ Transcribed Text:", stt_resp)
    original_text = stt_resp.transcript
    detected_lang = stt_resp.language_code
else:
    raise ValueError('No audio file - abort.')

🗣️ Transcribed Text: request_id='20250704_409b95aa-7e50-4e4b-9107-a81989a53751' transcript='केशव के घर में चार खिड़कियां हैं।\nकई लोग कुमार को पसंद करते हैं।\nतुम्हारे खरगोश का रंग सफेद है।\nआपकी गाय कल से यहां है।\nकल का खाना सुलेखा घी डालकर बनाएगी।\nअक्षय की खीर गरम हो गई।\nमैंने कल ख्वाब में एक खूबसूरत' timestamps=None diarized_transcript=None language_code='hi-IN'
CPU times: user 7.96 ms, sys: 10.5 ms, total: 18.5 ms
Wall time: 965 ms


## 🌐 Step 2 – Translate Text

In [13]:
%%time
# Example detected_lang and text
detected_lang = detected_lang if detected_lang != "unknown" else "hi-IN"
translated_texts = []
chunks = [original_text]  # You can replace this with actual chunking if needed

for idx, chunk in enumerate(chunks):
    resp = client.text.translate(
        input=chunk,
        source_language_code=detected_lang,
        target_language_code=TARGET_LANG,
        speaker_gender="Male",
        mode="formal",
        model="sarvam-translate:v1",
        enable_preprocessing=False,
    )
    print(f"Chunk {idx + 1}:\n", resp.translated_text, "\n")
    translated_texts.append(resp.translated_text)

translated_text = "\\n".join(translated_texts)
print("📝 Final Translation:", translated_text)

Chunk 1:
 ಕೇಶವ್ ಮನೆಗೆ ನಾಲ್ಕು ಕಿಟಕಿಗಳಿವೆ.
ಕುಮಾರ್ ಎಂದರೆ ಹಲವರಿಗೆ ಇಷ್ಟ.
ನಿಮ್ಮ ಮೊಲ ಬಿಳಿ ಬಣ್ಣದಲ್ಲಿದೆ.
ನಿಮ್ಮ ಹಸು ನಿನ್ನೆ ಇಲ್ಲಿಗೆ ಬಂದಿದೆ.
ನಿನ್ನೆಯ ಅಡುಗೆಯನ್ನು ತುಪ್ಪದಲ್ಲಿ ಅವಳು ಮಾಡ್ತಾಳೆ.
ಅಕ್ಷಯ್ ಖೀರ್ ಬಿಸಿಯಾಗಿದೆ.
ನಿನ್ನೆ ನನಗೆ ಒಂದು ಸುಂದರವಾದ ಕನಸು ಬಿತ್ತು. 

📝 Final Translation: ಕೇಶವ್ ಮನೆಗೆ ನಾಲ್ಕು ಕಿಟಕಿಗಳಿವೆ.
ಕುಮಾರ್ ಎಂದರೆ ಹಲವರಿಗೆ ಇಷ್ಟ.
ನಿಮ್ಮ ಮೊಲ ಬಿಳಿ ಬಣ್ಣದಲ್ಲಿದೆ.
ನಿಮ್ಮ ಹಸು ನಿನ್ನೆ ಇಲ್ಲಿಗೆ ಬಂದಿದೆ.
ನಿನ್ನೆಯ ಅಡುಗೆಯನ್ನು ತುಪ್ಪದಲ್ಲಿ ಅವಳು ಮಾಡ್ತಾಳೆ.
ಅಕ್ಷಯ್ ಖೀರ್ ಬಿಸಿಯಾಗಿದೆ.
ನಿನ್ನೆ ನನಗೆ ಒಂದು ಸುಂದರವಾದ ಕನಸು ಬಿತ್ತು.
CPU times: user 4.95 ms, sys: 4.52 ms, total: 9.47 ms
Wall time: 1.1 s


## 🔊 Step 3 – Generate Speech (Bulbul)

In [15]:
tts_resp = client.text_to_speech.convert(
    text=translated_text,
    target_language_code=f"{TARGET_LANG}" if TARGET_LANG!='en' else "en-IN",
    speaker="anushka",             # female voice; change as desired
    enable_preprocessing=True
)
# Play inline (Colab/Jupyter)
play(tts_resp)
# Save to file
save(tts_resp, "output_audio.wav")
print("💾 Saved to output_audio.wav")

💾 Saved to output_audio.wav


## 🔡 (Optional) Transliteration Examples

In [16]:
hin_text = "मुझे कल 9:30am को appointment है"
print("Original Hindi:", hin_text)

# Indic → Roman
roman = client.text.transliterate(
    input=hin_text,
    source_language_code="hi-IN",
    target_language_code="en-IN",
    spoken_form=True
).transliterated_text
print("Romanised:", roman)

# Roman → Indic
back = client.text.transliterate(
    input=roman,
    source_language_code="hi-IN",
    target_language_code="hi-IN",
    spoken_form=True
).transliterated_text
print("Back to Hindi:", back)

# Native vs English numerals speech
native_num = client.text.transliterate(
    input=hin_text,
    source_language_code="hi-IN",
    target_language_code="hi-IN",
    spoken_form=True,
    numerals_format="native"
).transliterated_text
print("Native numerals:", native_num)

eng_num = client.text.transliterate(
    input=hin_text,
    source_language_code="hi-IN",
    target_language_code="hi-IN",
    spoken_form=True,
    spoken_form_numerals_language="english"
).transliterated_text
print("English-style numerals:", eng_num)

Original Hindi: मुझे कल 9:30am को appointment है
Romanised: Muale ko kal 9:30am ko appointment hai
Back to Hindi: मुआले को कल साढ़े नौ बजे ए.एम को अपॉइंटमेंट है
Native numerals: मुझे कल सुबह साढ़े नौ बजे अपॉइंटमेंट है
English-style numerals: मुझे कल नाइन थर्टी ए एम को अपॉइंटमेंट है
